In [38]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
dataset_path = os.listdir('Dataset/archive/Train')

label_types = os.listdir('Dataset/archive/Train')
print (label_types)  

['NonViolence', 'Violence']


In [39]:
rooms = []

for item in dataset_path:
 all_rooms = os.listdir('Dataset/archive/Train' + '/' +item)

 for room in all_rooms:
    rooms.append((item, str('Dataset/archive/Train' + '/' +item) + '/' + room))
         
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

           tag                                   video_name
0  NonViolence   Dataset/archive/Train/NonViolence/NV_1.mp4
1  NonViolence  Dataset/archive/Train/NonViolence/NV_10.mp4
2  NonViolence  Dataset/archive/Train/NonViolence/NV_11.mp4
3  NonViolence  Dataset/archive/Train/NonViolence/NV_12.mp4
4  NonViolence  Dataset/archive/Train/NonViolence/NV_13.mp4
         tag                               video_name
95  Violence  Dataset/archive/Train/Violence/V_50.mp4
96  Violence   Dataset/archive/Train/Violence/V_6.mp4
97  Violence   Dataset/archive/Train/Violence/V_7.mp4
98  Violence   Dataset/archive/Train/Violence/V_8.mp4
99  Violence   Dataset/archive/Train/Violence/V_9.mp4


In [40]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [41]:
dataset_path = os.listdir('Dataset/archive/Test')
print(dataset_path)

room_types = os.listdir('Dataset/archive/Test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 all_rooms = os.listdir('Dataset/archive/Test' + '/' +item)

 for room in all_rooms:
    rooms.append((item, str('Dataset/archive/Test' + '/' +item) + '/' + room))
        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df.to_csv('test.csv')

['NonViolence', 'Violence']
Types of activities found:  2
           tag                                   video_name
0  NonViolence  Dataset/archive/Test/NonViolence/NV_901.avi
1  NonViolence  Dataset/archive/Test/NonViolence/NV_902.avi
2  NonViolence  Dataset/archive/Test/NonViolence/NV_903.avi
3  NonViolence  Dataset/archive/Test/NonViolence/NV_904.avi
4  NonViolence  Dataset/archive/Test/NonViolence/NV_905.avi
         tag                               video_name
15  Violence  Dataset/archive/Test/Violence/V_906.mp4
16  Violence  Dataset/archive/Test/Violence/V_907.mp4
17  Violence  Dataset/archive/Test/Violence/V_908.mp4
18  Violence  Dataset/archive/Test/Violence/V_909.mp4
19  Violence  Dataset/archive/Test/Violence/V_910.mp4


In [42]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [43]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 100
Total videos for testing: 20


,Unnamed: 0,video_name,tag
53,53,Dataset/archive/Train/Violence/V_12.mp4,Violence
46,46,Dataset/archive/Train/NonViolence/NV_6.mp4,NonViolence
34,34,Dataset/archive/Train/NonViolence/NV_40.mp4,NonViolence
84,84,Dataset/archive/Train/Violence/V_40.mp4,Violence
69,69,Dataset/archive/Train/Violence/V_27.mp4,Violence
36,36,Dataset/archive/Train/NonViolence/NV_42.mp4,NonViolence
14,14,Dataset/archive/Train/NonViolence/NV_22.mp4,NonViolence
60,60,Dataset/archive/Train/Violence/V_19.mp4,Violence
7,7,Dataset/archive/Train/NonViolence/NV_16.mp4,NonViolence
4,4,Dataset/archive/Train/NonViolence/NV_13.mp4,NonViolence


In [44]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
        
    return np.array(frames)

In [45]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [46]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['NonViolence', 'Violence']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [47]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [48]:
def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    labels = df["tag"].values
    
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    for idx, path in enumerate(video_paths):
        frames = load_video(path)
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1 

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

1/1 [==============================] - 0s 115ms/step
Frame features in train set: (100, 20, 2048)
Frame masks in train set: (100, 20)
train_labels in train set: (100, 1)
test_labels in train set: (20, 1)


In [49]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
3/3 [==============================] - ETA: 0s - loss: 0.7623 - accuracy: 0.4429
Epoch 1: val_loss improved from inf to 1.26045, saving model to ./tmp\video_classifier
3/3 [==============================] - 15s 2s/step - loss: 0.7623 - accuracy: 0.4429 - val_loss: 1.2604 - val_accuracy: 0.0000e+00
Epoch 2/30
3/3 [==============================] - ETA: 0s - loss: 0.6037 - accuracy: 0.7000
Epoch 2: val_loss improved from 1.26045 to 1.19191, saving model to ./tmp\video_classifier
3/3 [==============================] - 1s 299ms/step - loss: 0.6037 - accuracy: 0.7000 - val_loss: 1.1919 - val_accuracy: 0.0000e+00
Epoch 3/30
3/3 [==============================] - ETA: 0s - loss: 0.6032 - accuracy: 0.7000
Epoch 3: val_loss improved from 1.19191 to 1.14053, saving model to ./tmp\video_classifier
3/3 [==============================] - 1s 324ms/step - loss: 0.6032 - accuracy: 0.7000 - val_loss: 1.1405 - val_accuracy: 0.0000e+00
Epoch 4/30
3/3 [==============================] - ETA: 0s 

In [91]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(path)
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]
    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return probabilities

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

probs = sequence_prediction(test_video)

Test video path: Dataset/archive/Test/Violence/V_908.mp4
1/1 [==============================] - 0s 37ms/step
  Violence: 64.91%
  NonViolence: 35.09%
[0.35092115 0.64907885]


In [83]:
# for hate speech
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [84]:
data = pd.read_csv("DataSet/data.csv")
x = data.iloc[:,6].values
y = data.iloc[:,5].values

In [85]:
def pre_process_data(tweet):
    new_tweet = [];
    for word in tweet.split(" "):
        word = word.lower()
        word = re.sub('^(http|https):[a-zA-Z0-9-/_\.]*$', '', word)
        word = re.sub('^(@|#|&)+[a-zA-Z0-9]*:?;?$', '', word)
        word = re.sub('[^a-zA-Z]', '', word)
        new_tweet.append(word)
    tweet = " ".join(new_tweet)
    tweet_tokens = word_tokenize(tweet)
    filtered_tweets = [w for w in tweet_tokens if not w in stop_words]
    filtered_tweets = [lemmatizer.lemmatize(w) for w in filtered_tweets]
    return " ".join(new_tweet)

In [86]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

corpus = []
lemmatizer = WordNetLemmatizer()
for tweet in x:
  corpus.append(pre_process_data(tweet))

# making 0 and 1 = 1 (violence) and 2 = 0(nonviolence) because In the dataset the values are different

for i in range(len(y)):
  if y[i] == 0 or y[i] == 1:
    y[i] = 1
  else :
    y[i] = 0

x = corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
    
vect = TfidfVectorizer(ngram_range=(1,2)).fit(x)
x = vect.transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

def get_hate_speech_classifier():
    classifier = MultinomialNB()
    classifier.fit(x_train, y_train)
    return classifier
    
hate_speech_classifier = get_hate_speech_classifier()

In [89]:
y_pred = hate_speech_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

[[0.06324498 0.93675502]]


In [111]:
def predict_hate_speech(text):
    return hate_speech_classifier.predict_proba(text)[0]

In [115]:
def get_combained_prediction(text, path):
    if text != '':
        speech_pred = predict_hate_speech(pre_process_data(text))
    else :
        speech_pred = [0.5,0.5]
    video_pred = sequence_prediction(path)
    weights = [.6, .4]
    weighted_avg_pred = [(weights[0]*speech_pred[0]+weights[1]*video_pred[0])/2, (weights[0]*speech_pred[1]+weights[1]*video_pred[1])/2] 
    if weighted_avg_pred[0] >= weighted_avg_pred[1]:
        return label_types[0]
    else :
        return label_types[1]    
    

In [116]:
from moviepy.editor import *
import speech_recognition as sr

def get_text_from_video(path):
    audio_path = 'output_audio.wav'
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    video.close()
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        text = ""
    except sr.RequestError as e:
        text = ""
    if os.path.exists(audio_path):
        os.remove(audio_path)
    return text    


In [118]:
video_path = 'Dataset/archive/Test/Violence/V_905.mp4'
text = get_text_from_video(video_path)
print(get_combained_prediction(text, video_path))

MoviePy - Writing audio in output_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 42ms/step
  Violence: 66.27%
  NonViolence: 33.73%
[0.33725715 0.66274285]
Violence
